## CSV Preprocess

In [15]:
"""
! conda activate Seol
! pip install geopandas
! pip install shapely
"""

import pandas as pd
import os 
from geopandas import GeoDataFrame
import pickle
from shapely.geometry import Point

cur_path = os.getcwd() # 경로 설정

# subway_line= pd.read_csv('서울시 역사마스터 정보.csv', engine='python', encoding='utf-8')
subway_line= pd.read_csv('서울시 역사마스터 정보 최종.csv', engine='python', encoding='utf-8')

In [16]:
subway_line.head()

,역사_ID,역사명,호선,위도,경도
0,150,서울역,1호선,37.554648,126.972559
1,151,시청,1호선,37.564718,126.977108
2,152,종각,1호선,37.570161,126.982923
3,153,종로3가,1호선,37.571607,126.991806
4,154,종로5가,1호선,37.570926,127.001849


In [23]:
subway_line.describe

<bound method NDFrame.describe of      역사_ID     역사명     호선         위도          경도
0      150     서울역    1호선  37.554648  126.972559
1      151      시청    1호선  37.564718  126.977108
2      152      종각    1호선  37.570161  126.982923
3      153    종로3가    1호선  37.571607  126.991806
4      154    종로5가    1호선  37.570926  127.001849
..     ...     ...    ...        ...         ...
768   3223   모래내시장  인천2호선  37.455830  126.719298
769   3224      만수  인천2호선  37.454911  126.732094
770   3225    남동구청  인천2호선  37.448161  126.736939
771   3226   인천대공원  인천2호선  37.448769  126.752618
772   3227  운연(서창)  인천2호선  37.440127  126.759970

[773 rows x 5 columns]>

In [18]:
subway_line["역사정보"].head()

0    150
1    151
2    152
3    153
4    154
Name: 역사_ID, dtype: int64

In [19]:
list(subway_line.loc[(subway_line["역사명"] == '하계'), ["역사명", "위도", "경도"]].itertuples(name=None))

[(348, '하계', 37.636352, 127.06799)]

In [20]:
subway_line.loc[(subway_line["역사명"] == '충무로'), :]

,역사_ID,역사명,호선,위도,경도
161,321,충무로,3호선,37.561430,126.994072
211,423,충무로,4호선,37.561207,126.994080


## Excel Preprocessing

In [30]:
! conda activate Seol
! pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable


In [50]:
from openpyxl import load_workbook

load_wb = load_workbook("./서울 지하철 역간 거리 정보.xlsx", data_only=True)

In [52]:
seoul_underground = ['1호선', '2호선', '3호선', 
                     '4호선', '5호선', '6호선', '7호선', '8호선', '9호선']

In [51]:
load_ws = load_wb['1호선']
# print(load_ws.cell(3,2).value)

In [47]:
for line in seoul_underground:
    load_ws = load_wb[line]

number line name distance 
1 1호선 소요산 0 
2 1호선 동두천 2.5 
3 1호선 보산 1.6 
4 1호선 동두천중앙 1.4 
5 1호선 지행 1 
6 1호선 덕정 5.6 
7 1호선 덕계 2.9 
8 1호선 양주 5.3 


In [ ]:
get_cells = load_ws['A1':'D9']
    for row in get_cells:
        for cell in row:
            print(cell.value, end=' ')
        print("")

In [2]:
def find_all_paths(graph, start, end, path=[]):
        path = path + [start]
        if start == end:
            return [path]
        if start not in graph:
            return []
        paths = []
        for node in graph[start]:
            if node not in path:
                newpaths = find_all_paths(graph, node, end, path)
                for newpath in newpaths:
                    paths.append(newpath)
        return paths       

def min_path(graph, start, end):
    paths=find_all_paths(graph,start,end)
    mt=10**99
    mpath=[]
    print('\tAll paths:',paths)
    for path in paths:
        t=sum(graph[i][j] for i,j in zip(path,path[1::]))
        print('\t\tevaluating:',path, t)
        if t<mt: 
            mt=t
            mpath=path

    e1=' '.join('{}->{}:{}'.format(i,j,graph[i][j]) for i,j in zip(mpath,mpath[1::]))
    e2=str(sum(graph[i][j] for i,j in zip(mpath,mpath[1::])))
    print('Best path: '+e1+'   Total: '+e2+'\n')

if __name__ == "__main__":
    graph = {'A': {'B':5, 'C':4},
             'B': {'C':3, 'D':10},
             'C': {'D':12},
             'D': {'C':5, 'E':9},
             'E': {'F':8},
             'F': {'C':7}}
    min_path(graph,'A','E')
    min_path(graph,'A','D')
    min_path(graph,'A','F')

	All paths: [['A', 'B', 'C', 'D', 'E'], ['A', 'B', 'D', 'E'], ['A', 'C', 'D', 'E']]
		evaluating: ['A', 'B', 'C', 'D', 'E'] 29
		evaluating: ['A', 'B', 'D', 'E'] 24
		evaluating: ['A', 'C', 'D', 'E'] 25
Best path: A->B:5 B->D:10 D->E:9   Total: 24

	All paths: [['A', 'B', 'C', 'D'], ['A', 'B', 'D'], ['A', 'C', 'D']]
		evaluating: ['A', 'B', 'C', 'D'] 20
		evaluating: ['A', 'B', 'D'] 15
		evaluating: ['A', 'C', 'D'] 16
Best path: A->B:5 B->D:10   Total: 15

	All paths: [['A', 'B', 'C', 'D', 'E', 'F'], ['A', 'B', 'D', 'E', 'F'], ['A', 'C', 'D', 'E', 'F']]
		evaluating: ['A', 'B', 'C', 'D', 'E', 'F'] 37
		evaluating: ['A', 'B', 'D', 'E', 'F'] 32
		evaluating: ['A', 'C', 'D', 'E', 'F'] 33
Best path: A->B:5 B->D:10 D->E:9 E->F:8   Total: 32



## CSV 파일 기반 노선도 그래프 구축

: 상윤이가 하기로 함 (Firebase import/export w/ csv)

In [ ]:
! conda activate Seol
! pip install firebase_admin

In [38]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import json

cred = credentials.Certificate\
    ("freshroute-b533b-firebase-adminsdk-dr0b0-5c0276c696.json")

firebase_admin.initialize_app(cred, {
  'projectId': 'freshroute-b533b',
})

db = firestore.client()
doc_ref = db.collection("Subway").document("1002")

doc = doc_ref.get()
if doc.exists:
    print(f"Document data: {doc.to_dict()}")
else:
    print("No such document!")

ModuleNotFoundError: No module named 'firebase_admin'

# From Local json

In [39]:
import json
import pprint
import copy

with open("./data/bus.json", "r") as bus:
    map_bus = json.load(bus)
with open("./data/subway.json", "r") as subway:
    map_subway = json.load(subway)
with open("./data/subway_transfer.json", "r") as transfer:
    map_trans = json.load(transfer)

In [40]:
for tmp in map_subway:
    if map_subway[tmp]['name'] == '사당':
        print(tmp)

226
433


In [41]:
pp = pprint.PrettyPrinter(depth=4)
# pp.pprint(map_subway)
# pp.pprint(map_trans)
pp.pprint(map_subway['226'])
pp.pprint(map_subway['433'])
print("\n")
pp.pprint(map_trans['226'])
pp.pprint(map_trans['433'])
print("\n")
pp.pprint(map_bus['117000050'])

{'adj': {'0': {'dist': 1.6, 'id': '225', 'line': '2호선', 'name': '방배'},
         '1': {'dist': 1.7, 'id': '227', 'line': '2호선', 'name': '낙성대'}},
 'id': '226',
 'latitude': 37.476538,
 'line': '2호선',
 'longitude': 126.981544,
 'name': '사당'}
{'adj': {'0': {'dist': 1.6, 'id': '434', 'line': '4호선', 'name': '남태령'},
         '1': {'dist': 1.1, 'id': '432', 'line': '4호선', 'name': '이수'}},
 'id': '433',
 'latitude': 37.476955,
 'line': '4호선',
 'longitude': 126.981651,
 'name': '사당'}


{'id': '226',
 'line': '2호선',
 'name': '사당',
 'transferInfo': {'dist': 74, 'line': '4호선', 'name': '사당', 'time': '1분2초'}}
{'id': '433',
 'line': '4호선',
 'name': '사당',
 'transferInfo': {'dist': 74, 'line': '2호선', 'name': '사당', 'time': '1분2초'}}


{'adj': {'0': {'ARS_ID': '21172',
               'NODE_ID': 120000070,
               'ROUTE_ID': 100100521,
               'X좌표': 126.922015,
               'Y좌표': 37.458697,
               '노선명': '6515',
               '순번': 53,
               '정류소명': '산복터널.관악산휴먼시아2단지'},
  

In [50]:
map_bus_tmp = copy.deepcopy(map_bus)

# Redundant values
# 
# ARS_ID, mobile_number ()
# ID, NODE_ID 동일 ()
# ROUTE_ID, 노선명 1대1 대응

# pp.pprint(map_bus)


for tmp in map_bus_tmp:
    map_bus_tmp[tmp]['node_id'] = map_bus_tmp[tmp].pop('id')
    map_bus_tmp[tmp]['ars_id'] = map_bus_tmp[tmp].pop('mobile_number')
    try:
        for tmp2 in map_bus_tmp[tmp]['adj']:
            map_bus_tmp[tmp]['adj'][tmp2]['node_id'] = map_bus_tmp[tmp]['adj'][tmp2].pop('NODE_ID')
            map_bus_tmp[tmp]['adj'][tmp2]['latitude'] = map_bus_tmp[tmp]['adj'][tmp2].pop('Y좌표')
            map_bus_tmp[tmp]['adj'][tmp2]['longitude'] = map_bus_tmp[tmp]['adj'][tmp2].pop('X좌표')
            map_bus_tmp[tmp]['adj'][tmp2]['station_name'] = map_bus_tmp[tmp]['adj'][tmp2].pop('정류소명')
            map_bus_tmp[tmp]['adj'][tmp2]['bus_number'] = map_bus_tmp[tmp]['adj'][tmp2].pop('노선명')
            map_bus_tmp[tmp]['adj'][tmp2]['ars_id'] = map_bus_tmp[tmp]['adj'][tmp2].pop('ARS_ID')
            map_bus_tmp[tmp]['adj'][tmp2]['route_id'] = map_bus_tmp[tmp]['adj'][tmp2].pop('ROUTE_ID')
    except:
        print("",end='')
    
pp.pprint(map_bus_tmp)

{'117000050': {'adj': {'0': {'ars_id': '21172',
                             'bus_number': '6515',
                             'latitude': 37.458697,
                             'longitude': 126.922015,
                             'node_id': 120000070,
                             'route_id': 100100521,
                             'station_name': '산복터널.관악산휴먼시아2단지',
                             '순번': 53},
                       '1': {'ars_id': '18138',
                             'bus_number': '6515',
                             'latitude': 37.446768,
                             'longitude': 126.918953,
                             'node_id': 117000052,
                             'route_id': 100100521,
                             'station_name': '벽산아파트5단지',
                             '순번': 55},
                       '2': {'ars_id': '21172',
                             'bus_number': '5517',
                             'latitude': 37.458697,
                             'lo

                       '4': {'ars_id': '21221',
                             'bus_number': '6516',
                             'latitude': 37.4815094057,
                             'longitude': 126.9053800168,
                             'node_id': 120000119,
                             'route_id': 100100576,
                             'station_name': '골목시장앞',
                             '순번': 46},
                       '5': {'ars_id': '18193',
                             'bus_number': '6516',
                             'latitude': 37.4753270832,
                             'longitude': 126.9033549166,
                             'node_id': 117000107,
                             'route_id': 100100576,
                             'station_name': '독산3동주민센터',
                             '순번': 48}},
               'ars_id': 18195,
               'latitude': 37.4788389,
               'longitude': 126.9036963,
               'node_id': '117000109',
               'node_name

                             'route_id': 117900008,
                             'station_name': '벽산1단지입구',
                             '순번': 16},
                       '1': {'ars_id': '18764',
                             'bus_number': '금천01',
                             'latitude': 37.4500269983,
                             'longitude': 126.9167740089,
                             'node_id': 117900035,
                             'route_id': 117900008,
                             'station_name': '동일여고.금천사이언스큐브',
                             '순번': 18}},
               'ars_id': 18639,
               'latitude': 37.4499598,
               'longitude': 126.9181165,
               'node_id': '117900048',
               'node_name': '벽산6단지입구'},
 '117900053': {'adj': {'0': {'ars_id': '18590',
                             'bus_number': '금천01',
                             'latitude': 37.4517227734,
                             'longitude': 126.9205758327,
                             

                             'bus_number': '5536',
                             'latitude': 37.4955867471,
                             'longitude': 126.9115745553,
                             'node_id': 118000011,
                             'route_id': 100100268,
                             'station_name': '경남아너스빌아파트',
                             '순번': 47},
                       '9': {'ars_id': '19009',
                             'bus_number': '5536',
                             'latitude': 37.487149,
                             'longitude': 126.904837,
                             'node_id': 118000009,
                             'route_id': 100100268,
                             'station_name': '서울영림초등학교',
                             '순번': 49}},
               'ars_id': 19010,
               'latitude': 37.4908562,
               'longitude': 126.9078761,
               'node_id': '118000010',
               'node_name': '강남성심병원.대림성모병원'},
 '118000011': {'adj': {'0': {'a

                             'latitude': 37.50506,
                             'longitude': 126.91325,
                             'node_id': 118000610,
                             'route_id': 100100293,
                             'station_name': '신길파크자이아파트',
                             '순번': 21},
                       '6': {'ars_id': '19249',
                             'bus_number': '8641',
                             'latitude': 37.5013238443,
                             'longitude': 126.9100850037,
                             'node_id': 118000164,
                             'route_id': 114000002,
                             'station_name': '신풍역.백악관웨딩문화원앞',
                             '순번': 21},
                       '7': {'ars_id': '19238',
                             'bus_number': '8641',
                             'latitude': 37.50958,
                             'longitude': 126.910838,
                             'node_id': 118000153,
                      

...

                  'longitude': 126.9036414,
                  'node_id': 'SEB117000108',
                  'node_name': '독산3동주민센터'},
 'SEB117000109': {'ars_id': 18195,
                  'latitude': 37.4788389,
                  'longitude': 126.9036963,
                  'node_id': 'SEB117000109',
                  'node_name': '모두의학교.금천문화예술정보학교'},
 'SEB117000110': {'ars_id': 18196,
                  'latitude': 37.4786589,
                  'longitude': 126.9039351,
                  'node_id': 'SEB117000110',
                  'node_name': '모두의학교.금천문화예술정보학교'},
 'SEB117000111': {'ars_id': 18197,
                  'latitude': 37.4799497,
                  'longitude': 126.9055934,
                  'node_id': 'SEB117000111',
                  'node_name': '모두의학교.금천문화예술정보학교'},
 'SEB117000112': {'ars_id': 18198,
                  'latitude': 37.4806388,
                  'longitude': 126.9087572,
                  'node_id': 'SEB117000112',
                  'node_name': '관악구보훈회관.신림푸르지오'

 'SEB119000063': {'ars_id': 20156,
                  'latitude': 37.5121264,
                  'longitude': 126.9543267,
                  'node_id': 'SEB119000063',
                  'node_name': '상도터널노량진동'},
 'SEB119000064': {'ars_id': 20157,
                  'latitude': 37.5120063,
                  'longitude': 126.9536978,
                  'node_id': 'SEB119000064',
                  'node_name': '상도터널노량진동'},
 'SEB119000065': {'ars_id': 20158,
                  'latitude': 37.5045988,
                  'longitude': 126.9495037,
                  'node_id': 'SEB119000065',
                  'node_name': '상도터널상도동'},
 'SEB119000066': {'ars_id': 20159,
                  'latitude': 37.5038586,
                  'longitude': 126.949519,
                  'node_id': 'SEB119000066',
                  'node_name': '상도터널상도동'},
 'SEB119000067': {'ars_id': 20160,
                  'latitude': 37.5016271,
                  'longitude': 126.9486225,
                  'node_id': 'SEB119000067

 'SEB119900165': {'ars_id': 20814,
                  'latitude': 37.5069461,
                  'longitude': 126.9319982,
                  'node_id': 'SEB119900165',
                  'node_name': '대방목욕탕'},
 'SEB119900166': {'ars_id': 20813,
                  'latitude': 37.5063972,
                  'longitude': 126.9420033,
                  'node_id': 'SEB119900166',
                  'node_name': '새소망교회'},
 'SEB119900167': {'ars_id': 20811,
                  'latitude': 37.4894721,
                  'longitude': 126.9771578,
                  'node_id': 'SEB119900167',
                  'node_name': '시장입구'},
 'SEB119900168': {'ars_id': 20809,
                  'latitude': 37.5083531,
                  'longitude': 126.9471191,
                  'node_id': 'SEB119900168',
                  'node_name': '우성아파트'},
 'SEB119900169': {'ars_id': 20808,
                  'latitude': 37.4998232,
                  'longitude': 126.9267443,
                  'node_id': 'SEB119900169',
       

 'SEB120000109': {'ars_id': 21211,
                  'latitude': 37.4652824,
                  'longitude': 126.9196773,
                  'node_id': 'SEB120000109',
                  'node_name': '신림복지관앞'},
 'SEB120000110': {'ars_id': 21212,
                  'latitude': 37.465357,
                  'longitude': 126.919495,
                  'node_id': 'SEB120000110',
                  'node_name': '신림복지관앞'},
 'SEB120000111': {'ars_id': 21213,
                  'latitude': 37.467392,
                  'longitude': 126.920348,
                  'node_id': 'SEB120000111',
                  'node_name': '난우중학교입구'},
 'SEB120000112': {'ars_id': 21214,
                  'latitude': 37.4706671,
                  'longitude': 126.9197417,
                  'node_id': 'SEB120000112',
                  'node_name': '우림시장.난곡보건분소'},
 'SEB120000113': {'ars_id': 21215,
                  'latitude': 37.4727039,
                  'longitude': 126.9188803,
                  'node_id': 'SEB120000113',


 'SEB120900082': {'ars_id': 21502,
                  'latitude': 37.4634784,
                  'longitude': 126.9189072,
                  'node_id': 'SEB120900082',
                  'node_name': '난향동공영차고지'},
 'SEB120900083': {'ars_id': 21501,
                  'latitude': 37.4720765,
                  'longitude': 126.9110629,
                  'node_id': 'SEB120900083',
                  'node_name': '아카시아마을.민방위교육장'},
 'SEB120900084': {'ars_id': 21599,
                  'latitude': 37.4804996,
                  'longitude': 126.9148759,
                  'node_id': 'SEB120900084',
                  'node_name': '신림청소년독서실.신림체육센터'},
 'SEB120900085': {'ars_id': 21600,
                  'latitude': 37.4787543,
                  'longitude': 126.9575345,
                  'node_id': 'SEB120900085',
                  'node_name': '낙성대동시장입구'},
 'SEB120900086': {'ars_id': 21601,
                  'latitude': 37.4852937,
                  'longitude': 126.9341252,
                  'node_id'

## Node class 정의

In [25]:
class Node:
    def __init__(self, id=None, name=None, lat=None, long=None, adj=None):
        self.id = id
        self.name = name
        
        self.lat = lat
        self.long = long
        
        self.adj = []

        self.g = 0
        self.h = 0
        self.f = 0

    def __eq__(self, other):
        return self.lat == other.lat and self.long == other.long

## Heuristics

In [26]:
def heuristic(node, goal, D=1, D2=2 ** 0.5):
    dx = abs(node.lat - goal.lat)
    dy = abs(node.long - goal.long)
    return D * (dx + dy) + (D2 - 2 * D) * min(dx, dy)

 ## A-star class

In [54]:
# start = list(subway_line.loc[(subway_line["역사명"] == '하계'), ["역사명", "위도", "경도"]].itertuples(name=None))
# end = list(subway_line.loc[(subway_line["역사명"] == '충무로'), ["역사명", "위도", "경도"]].itertuples(name=None))

start = list(subway_line.loc[(subway_line["역사명"] == '하계'), ["역사명", "위도", "경도"]].to_records()[0])
end = list(subway_line.loc[(subway_line["역사명"] == '충무로'), ["역사명", "위도", "경도"]].to_records()[0])

startNode = Node(*start)
endNode = Node(*end)
openList = []
closedList = []

print(startNode.id, endNode.id)
openList.append(startNode)
# print(openList)

348 161


In [55]:
while openList:
    
    currentNode = openList[0]
    currentIdx = 0
    
    # print(currentNode, currentIdx)

    for index, item in enumerate(openList):
        if item.f < currentNode.f:
            currentNode = item
            currentIdx = index

    openList.pop(currentIdx)
    closedList.append(currentNode)

    if currentNode == endNode:
        path = []
        current = currentNode
        while current is not None:
            path.append(current.lat, current.long)
            current = current.parent
        print(path[::-1])

    children = []

    for newID in currentNode.adj:

        new_node = Node(*(list(subway_line.loc[(subway_line["역사_ID"] == newID), ["역사명", "위도", "경도"]].to_records()[0])))
        children.append(new_node)

    for child in children:

        if child in closedList:
            continue

        # f, g, h값 업데이트
        child.g = currentNode.g + 1
        child.h = ((child.lat - endNode.lat) **
                   2) + ((child.lat - endNode.lat) ** 2)
        # child.h = heuristic(child, endNode) 다른 휴리스틱
        # print("position:", child.position) 거리 추정 값 보기
        # print("from child to goal:", child.h)

        child.f = child.g + child.h

        if len([openNode for openNode in openList
                if child == openNode and child.g > openNode.g]) > 0:
            continue

        openList.append(child)

<__main__.Node object at 0x00000206821FDDF0> 0


## Backbone code

In [1]:
class Node:
    def __init__(self, parent=None, position=None):
        self.parent = parent
        self.position = position

        self.g = 0
        self.h = 0
        self.f = 0

    def __eq__(self, other):
        return self.position == other.position

def heuristic(node, goal, D=1, D2=2 ** 0.5):
    dx = abs(node.position[0] - goal.position[0])
    dy = abs(node.position[1] - goal.position[1])
    return D * (dx + dy) + (D2 - 2 * D) * min(dx, dy)


def aStar(maze, start, end):
    
    startNode = Node(None, start)
    endNode = Node(None, end)

    openList = []
    closedList = []

    openList.append(startNode)

    while openList:

        currentNode = openList[0]
        currentIdx = 0

        for index, item in enumerate(openList):
            if item.f < currentNode.f:
                currentNode = item
                currentIdx = index

        openList.pop(currentIdx)
        closedList.append(currentNode)

        if currentNode == endNode:
            path = []
            current = currentNode
            while current is not None:
                # x, y = current.position
                path.append(current.position)
                current = current.parent
            return path[::-1]  # reverse

        children = []
        
        # 인접한 xy좌표 전부
        for newPosition in [(0, -1), (0, 1), (-1, 0), (1, 0), (-1, -1), (-1, 1), (1, -1), (1, 1)]:

            # 노드 위치 업데이트
            nodePosition = (
                currentNode.position[0] + newPosition[0],  # X
                currentNode.position[1] + newPosition[1])  # Y
                
            # 미로 maze index 범위 안에 있어야함
            within_range_criteria = [
                nodePosition[0] > (len(maze) - 1),
                nodePosition[0] < 0,
                nodePosition[1] > (len(maze[len(maze) - 1]) - 1),
                nodePosition[1] < 0,
            ]

            if any(within_range_criteria):  # 하나라도 true면 범위 밖임
                continue

            # 장애물이 있으면 다른 위치 불러오기
            if maze[nodePosition[0]][nodePosition[1]] != 0:
                continue

            new_node = Node(currentNode, nodePosition)
            children.append(new_node)

        # 자식들 모두 loop
        for child in children:

            # 자식이 closedList에 있으면 continue
            if child in closedList:
                continue

            # f, g, h값 업데이트
            child.g = currentNode.g + 1
            child.h = ((child.position[0] - endNode.position[0]) **
                       2) + ((child.position[1] - endNode.position[1]) ** 2)
            # child.h = heuristic(child, endNode) 다른 휴리스틱
            # print("position:", child.position) 거리 추정 값 보기
            # print("from child to goal:", child.h)
            
            child.f = child.g + child.h

            if len([openNode for openNode in openList
                    if child == openNode and child.g > openNode.g]) > 0:
                continue
                    
            openList.append(child)


def main():
    # 1은 장애물
    maze = [[0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

    start = (0, 0)
    end = (7, 6)

    path = aStar(maze, start, end)
    print(path)


if __name__ == '__main__':
    main()



[(0, 0), (1, 1), (2, 2), (3, 3), (4, 3), (5, 4), (6, 5), (7, 6)]


## Class